In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import os
import glob
import time

import pickle

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision

import pytorch_lightning as pl

import sklearn
import functools
import operator

import sys

import optuna

sys.path.insert(1,"/home/sbulusu/qcd_ml/neural_networks/libs/")
sys.path.insert(1,"/home/sbulusu/qcd_ml/neural_networks/libs/pytorch-conv4d/")

import ym_hdf5_dataset
import custom_torch_net_class_lightning
import utils
import conv4d

from tqdm.notebook import tqdm

In [2]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")
    print("Utilizing CUDA")
else:
    device = torch.device("cpu")
    print("Utilizing CPU")
    

Utilizing CUDA


In [3]:
random_seed = 42
np.random.seed(random_seed)
torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
torch.backends.cudnn.enabled = False
torch.backends.cudnn.deterministic = True

In [4]:
"""
Load Data
"""
#conf_file_dir = "/media/data/sbulusu/datasets/kl_config/"
conf_file_dir = "/media/data/sbulusu/datasets/ym_datasets/"

file_format_list = ["test_4_8p3.hdf5"]

lat_size = [4,8,8,8]
dim = len(lat_size)
c = dim + 6
output_size = np.concatenate((lat_size, [-1]))
print(output_size)

train_label_names = ["P"]

dataset = ym_hdf5_dataset.ym_hdf5_dataset(conf_file_dir, file_format_list, lat_size, output_size, transform=None, device=device)

[ 4  8  8  8 -1]


first example loaded
{'P': 0.010422191, 'beta': 0.1, 'trW': array([[ 0.58349127+0.j, -1.017543  +0.j, -0.06921043+0.j,
        -1.419396  +0.j, -0.05574272+0.j,  0.87641346+0.j],
       [-1.7720646 +0.j, -1.5833766 +0.j,  1.3895096 +0.j,
        -1.0107304 +0.j, -0.12213898+0.j,  0.6229094 +0.j],
       [ 0.6368654 +0.j, -0.16945331+0.j, -1.5695194 +0.j,
        -1.6255391 +0.j,  0.28061134+0.j,  0.98804086+0.j],
       ...,
       [ 1.2385771 +0.j, -0.90888715+0.j, -1.7265443 +0.j,
         0.9174739 +0.j, -0.63691056+0.j,  0.46253422+0.j],
       [-0.3219289 +0.j,  1.2537234 +0.j,  1.1477227 +0.j,
        -0.64215916+0.j,  0.93961644+0.j, -1.393872  +0.j],
       [ 1.4701228 +0.j,  1.0082756 +0.j, -0.13107477+0.j,
        -0.14585751+0.j, -1.0367914 +0.j, -0.47122845+0.j]],
      dtype=complex64), 'u': array([[[[-0.28234136-0.73234135j, -0.4979092 -0.3688424j ],
         [ 0.4979092 -0.3688424j , -0.28234136+0.73234135j]],

        [[-0.19623522-0.31067726j,  0.33836865+0.86629826j],

      dtype=complex64), 'y': 0}
Read conf file number 0 /media/data/sbulusu/datasets/ym_datasets/test_4_8p3.hdf5

last example loaded:
{'P': -0.73333776, 'beta': 6.0, 'trW': array([[1.8593231+0.j, 1.3474445+0.j, 1.6442561+0.j, 1.626732 +0.j,
        1.4145662+0.j, 1.9127969+0.j],
       [1.9853619+0.j, 1.7971154+0.j, 1.7459404+0.j, 1.8987795+0.j,
        1.875567 +0.j, 1.9932044+0.j],
       [1.9521285+0.j, 1.6290618+0.j, 1.7158161+0.j, 1.9311266+0.j,
        1.5421947+0.j, 1.91811  +0.j],
       ...,
       [1.7009405+0.j, 1.3165945+0.j, 1.8595853+0.j, 1.6361184+0.j,
        1.7038369+0.j, 1.8042083+0.j],
       [1.9282148+0.j, 1.5456533+0.j, 1.5760418+0.j, 1.3350972+0.j,
        1.1928371+0.j, 1.8592515+0.j],
       [1.4567833+0.j, 1.8670393+0.j, 1.8928581+0.j, 1.7633663+0.j,
        1.9731896+0.j, 1.8960235+0.j]], dtype=complex64), 'u': array([[[[-1.22250117e-01+0.25776985j, -3.03027719e-01+0.9092754j ],
         [ 3.03027719e-01+0.9092754j , -1.22250117e-01-0.25776985j]],

        

      dtype=complex64), 'y': 1}


In [5]:
target_attributes = ["P"]
output_attributes = target_attributes

dataset.train_label_names = target_attributes

In [6]:
sample = dataset[1]
conf = sample[0]
label = sample[1]
print(label)
print(conf.shape)
print(conf.dtype)

tensor([-0.0204])
(4, 8, 8, 8, 80)
float32


In [7]:

input_size = list(conf.shape)
print(input_size)

[4, 8, 8, 8, 80]


In [8]:
"""Manually set network structure"""
"""
    This list can be loaded into the constructor of the Net neural network class, to automatically generate the network structure
    type = pointer to the layer function'
    layer_pars = parameters which must be given to the layer function in order to initialize it
    act_func = activation function to be applied directly after feeding to the corresponding layer
    dropout = certain neurons cna be dropped out if specified
"""

cnn_struct = []
#input_size = dataset.output_size
input_size = np.array(input_size)
#print(f"input size {input_size}")
#target_size = len(target_attributes)
#print(f"target size {target_size}")
#output_size = target_size
output_size = len(target_attributes)

i0 = input_size[0]

#[ [[in_channels, out_channels], [kernel_size], [stride], padding], ... ]
kernel_pars = [
    [[i0,160],[3,3,3,3],1,0],
    [[160,320],[2,2,2,2],1,0],
    [[320,320],[1,4,4,4],1,0],
    [[320,320],[1,2,2,2],1,0],
]



#cnn_struct.append( {"type": layer_type, "layer_pars": {"in_channels": kernel_par[0][0], "out_channels": kernel_par[0][1], "kernel_size": kernel_par[1], "stride": kernel_par[2], "padding": kernel_par[3], "bias": True}} )
#cnn_struct.append( {"type": utils.NpSplitReImToChannel, "layer_pars": {"channel_axis": -1}} )
"""0 axis is batch axis!!!"""
cnn_struct.append( {"type": utils.PermuteAxes, "layer_pars": {"perm": [4,0,1,2,3]}} )

for i, kernel_par in enumerate(kernel_pars):
    layer_type = nn.Conv2d
    cnn_struct.append( {"type": conv4d.Conv4d,
                        "layer_pars": {"in_channels": kernel_par[0][0], "out_channels": kernel_par[0][1],
                                       "kernel_size": kernel_par[1],
                                        "stride": kernel_par[2], "padding": kernel_par[3],
                                       "bias": True}} )
    #fixed_net_struct.append( {"type": nn.PReLU, "layer_pars": {}} )
    cnn_struct.append( {"type": nn.ReLU, "layer_pars": {}} )
    

conv_sizes = utils.calc_layer_sizes(input_size, cnn_struct)
print(conv_sizes)

[array([ 4,  8,  8,  8, 80]), array([80,  4,  8,  8,  8]), [160, 2, 6, 6, 6], [160, 2, 6, 6, 6], [320, 1, 5, 5, 5], [320, 1, 5, 5, 5], [320, 1, 2, 2, 2], [320, 1, 2, 2, 2], [320, 1, 1, 1, 1], [320, 1, 1, 1, 1]]


In [9]:
fc_input_size = np.product(conv_sizes[-1])
print(fc_input_size)
dense1_struct = []
dense2_struct = []

dense1_struct.append( {"type": nn.Flatten, "layer_pars": {"start_dim": 1}} )
#fixed_net_struct.append( {"type": utils.Reshape, "layer_pars": {"new_shape": [fc_input_size]}} )
dense1_struct.append( {"type": nn.Linear, "layer_pars": {"in_features": fc_input_size, "out_features": 1}} )

layer_sizes = utils.calc_layer_sizes(fc_input_size, dense1_struct)
print(layer_sizes)

320
[320, 320, 1]


In [10]:
"""
HYPERPARAMETERS
"""
epochs = 500

"""create list of parameters manually"""

hyper_parameters = {}
hyper_parameters["name"] = "CustomNet"

hyper_parameters["cnn_struct"] = cnn_struct
hyper_parameters["classifier1_struct"] = dense1_struct
hyper_parameters["classifier2_struct"] = dense2_struct
hyper_parameters["input_size"] = input_size

hyper_parameters["fc_input_size"] = fc_input_size
hyper_parameters["output_size"] = [output_size]
#hyper_parameters["device"] = device

#hyper_parameters["dataset"] = dataset

hyper_parameters["random_seed"] = random_seed
hyper_parameters["epochs"] = epochs

hyper_parameters["bs"] = 32 
hyper_parameters["lr"] = 0.0001

#hyper_parameters["loss_func"] = nn.CrossEntropyLoss
#hyper_parameters["loss_func"] = nn.BCELoss
hyper_parameters["loss"] = nn.MSELoss
hyper_parameters["loss_kwargs"] = {}
hyper_parameters["optimizer"] = optim.Adam
hyper_parameters["optimizer_kwargs"] = {}

In [11]:
"""regular training with all data"""
batch_size = 16
val_method = "holdout"
split_ratio = {"train":0.7, "test":0.2, "val":0.1}
split_indices = utils.load_split_indices(dataset=dataset, batch_size=batch_size, method=val_method, method_pars=split_ratio, shuffle=True, random_seed=random_seed)



size of test set :501

size of val set :215

size of train set :1284



In [12]:
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning import Trainer, Callback

# copy class from https://github.com/optuna/optuna/blob/25f578c6d853a396f5c3b8817ace1ff74e6c7300/optuna/integration/pytorch_lightning.py
class PyTorchLightningPruningCallback(EarlyStopping):

    def __init__(self, trial, monitor):
        # type: (optuna.trial.Trial, str) -> None

        super(PyTorchLightningPruningCallback, self).__init__(monitor=monitor)

        self._trial = trial
        self._monitor = monitor

    def on_epoch_end(self, trainer, pl_module):
        # type: (Trainer, LightningModule) -> None
        logs = trainer.callback_metrics
        epoch = pl_module.current_epoch
        current_score = logs.get(self._monitor)
        if current_score is None:
            return
        self._trial.report(current_score, step=epoch)
        if self._trial.should_prune():
            message = "Trial was pruned at epoch {}.".format(epoch)
            raise optuna.exceptions.TrialPruned(message)

#at the end of validation, save the validation loss via this callback class
class MetricsCallback(pl.Callback):
    """PyTorch Lightning metric callback"""

    def __init__(self):
        super().__init__()
        self.metrics = []

    def on_validation_end(self, trainer, pl_module):
        self.metrics.append(trainer.callback_metrics)

In [13]:
lr_min = 10**(-7)
lr_max = 10**(-3)

bs_categories = [8, 16, 32, 64]

#objective to optimize with optuna
def objective(trial):
    #create ranges for adjustable parameters
    lr_range = trial.suggest_loguniform("lr", lr_min, lr_max)
    
    bs_range = trial.suggest_categorical("bs", bs_categories)
    
    #add them to all hyper parameters
    hyper_parameters["lr"] = lr_range
    hyper_parameters["bs"] = bs_range
    
    #create network instance
    model = custom_torch_net_class_lightning.CustomNet(hyper_parameters)
    
    hparams = model.hparams
    
    model.prepare_dataset_splits(dataset, split_indices)

    model.to(device)

    #train_loader = model.train_dataloader()
    #val_loader = model.val_dataloader()
    
    model.configure_loss()
    
    #create checkpoint callback function to save best model based on metric (monitor)
    checkpoint_callback = ModelCheckpoint(
        filepath= os.getcwd() + '/pl_checkpoints/{}-{:03d}'.format(model.hparams['name'], trial.number) + '-{epoch:03d}-{val_loss:.5E}',
        save_top_k=1,
        verbose=False,
        monitor='val_loss',
        mode='min',
        prefix=''
    )
    
    #create callback function to save validation loss
    metrics_callback = MetricsCallback()
    
    #logging with TensorBoard
    tb_logger = TensorBoardLogger("lightning_logs", name=hparams['name'])
    
    #lightning trainer instance
    #with callback functions for checkpoints, validation loss and early stopping
    trainer = pl.Trainer(
        logger=tb_logger,
        gpus=1,
        max_epochs=hyper_parameters["epochs"],
        checkpoint_callback=checkpoint_callback,
        callbacks=[metrics_callback],
        early_stop_callback=PyTorchLightningPruningCallback(trial, monitor="val_loss")
    )
    
    #training procedure
    trainer.fit(model)
    
    # collect all val_losses and return minimum (or average)
    val_losses = []
    for d in metrics_callback.metrics:
        if 'val_loss' in d:
            val_losses.append(d['val_loss'])
            
    target_metric = np.min(val_losses)
    
    return target_metric

In [14]:
n_trials = 2
study = optuna.create_study()
study.optimize(objective, n_trials=n_trials)

INFO:lightning:GPU available: True, used: True
INFO:lightning:VISIBLE GPUS: 0
INFO:lightning:
   | Name                         | Type        | Params
---------------------------------------------------------
0  | cnn                          | Net         | 8 M   
1  | cnn.layers                   | ModuleList  | 8 M   
2  | cnn.layers.0                 | PermuteAxes | 0     
3  | cnn.layers.1                 | Conv4d      | 52 K  
4  | cnn.layers.1.conv3d_layers   | ModuleList  | 52 K  
5  | cnn.layers.1.conv3d_layers.0 | Conv3d      | 17 K  
6  | cnn.layers.1.conv3d_layers.1 | Conv3d      | 17 K  
7  | cnn.layers.1.conv3d_layers.2 | Conv3d      | 17 K  
8  | cnn.layers.2                 | ReLU        | 0     
9  | cnn.layers.3                 | Conv4d      | 819 K 
10 | cnn.layers.3.conv3d_layers   | ModuleList  | 819 K 
11 | cnn.layers.3.conv3d_layers.0 | Conv3d      | 409 K 
12 | cnn.layers.3.conv3d_layers.1 | Conv3d      | 409 K 
13 | cnn.layers.4                 | ReLU        | 

torch.Size([64, 4, 8, 8, 8, 80])
torch.Size([64, 4, 8, 8, 8, 80])
torch.Size([64, 80, 4, 8, 8, 8])


[W 2020-04-30 13:10:24,714] Setting status of trial#0 as TrialState.FAIL because of the following error: RuntimeError('Given groups=1, weight of size 160 4 3 3 3, expected input[64, 80, 8, 8, 8] to have 4 channels, but got 80 channels instead')
Traceback (most recent call last):
  File "/home/sbulusu/anaconda3/envs/ml/lib/python3.7/site-packages/optuna/study.py", line 677, in _run_trial
    result = func(trial)
  File "<ipython-input-13-8eb03b83fb06>", line 59, in objective
    trainer.fit(model)
  File "/home/sbulusu/anaconda3/envs/ml/lib/python3.7/site-packages/pytorch_lightning/trainer/trainer.py", line 704, in fit
    self.single_gpu_train(model)
  File "/home/sbulusu/anaconda3/envs/ml/lib/python3.7/site-packages/pytorch_lightning/trainer/distrib_parts.py", line 477, in single_gpu_train
    self.run_pretrain_routine(model)
  File "/home/sbulusu/anaconda3/envs/ml/lib/python3.7/site-packages/pytorch_lightning/trainer/trainer.py", line 843, in run_pretrain_routine
    False)
  File "/

RuntimeError: Given groups=1, weight of size 160 4 3 3 3, expected input[64, 80, 8, 8, 8] to have 4 channels, but got 80 channels instead

In [ ]:
study.best_params

In [ ]:
study.best_trial
pickle.dump(study, open("study.pickle"), "wb")

In [ ]:
trial_coords = np.zeros((len(study.trials),2))

trial_metrics = np.zeros((len(study.trials)))

for trial_i, trial in enumerate(study.trials):
    par_i = 0
    for par_name in trial.params:
        trial_coords[trial_i, par_i] = trial.params[par_name]
        trial_metrics[trial_i] = trial.value
        par_i +=1
        
plt.scatter(trial_coords[:,0], trial_coords[:,1], c=trial_metrics, cmap="RdBu")

In [ ]:
checkpoint_path = "pl_checkpoints/CustomNet-009-epoch=992-val_loss=2.11364E-06.ckpt"

In [ ]:
for hpar in study.best_params:
#for hpar in best_params:
    print(hpar)
    hyper_parameters[hpar] = study.best_params[hpar]
    #hyper_parameters[hpar] = best_params[hpar]
print(hyper_parameters)

In [ ]:
hyper_parameters["cnn_struct"]

In [ ]:
#best_model = custom_torch_net_class_lightning.CustomNet.load_from_checkpoint(checkpoint_path,hyper_parameters)
best_model = custom_torch_net_class_lightning.CustomNet.load_from_checkpoint(checkpoint_path)

best_model.prepare_dataset_splits(dataset, split_indices)

best_model.to(device)

best_model.configure_loss()

best_model.eval()

In [ ]:
"""
Load Test Data
"""
net = best_model
net.eval()
print(f"test dataset size: {len(test_indices)}")

input_size = net.input_size
output_size = net.output_size
model_input_shape = tuple(np.concatenate(([-1],input_size)))
print(model_input_shape)

#net_outputs = np.zeros( (len(test_indices)) )
net_outputs = np.zeros( (len(test_indices), len(output_attributes)) )
#labels = np.zeros( (len(test_indices)) )
labels = np.zeros( (len(test_indices), len(output_attributes)) )
mus = np.zeros( (len(test_indices)) )

for i in tqdm(range(len(test_indices))):
#for i in tqdm(range(len(val_indices))):
    
    test_index = test_indices[i]
    conf_lat_links, label = dataset.get_conf(test_index)
    mus[i] = dataset.data[test_index]["mu"].detach().cpu().numpy()
    net_outputs[i] = net(conf_lat_links.view(model_input_shape).to(device).float()).detach().cpu().numpy()
    #output = net(conf_lat_links.view(model_input_shape).float())
    #labels[i] = label.detach().cpu().numpy()
    labels[i] = np.array(label)
    

In [ ]:
unique_mus = np.unique(mus)
num_mus = len(unique_mus)

In [ ]:
##save the observable values in a dictionary
obs_mu_dict = {}
for target_name in target_attributes:
    obs_mu_dict[target_name] = {"label" : [],"pred" : []}

In [ ]:

obs_label = []
obs_pred = []
for mu_val in unique_mus:
    mu_loc = np.where(mus == mu_val)[0]
    obs_label_mu = np.array(labels[mu_loc,0])
    obs_pred_mu = np.array(net_outputs[mu_loc,0])
    
    obs_label.append(obs_label_mu)    
    obs_pred.append(obs_pred_mu)

    

In [ ]:
##to be able to plot the observables against mu, they have to be sorted with respect to it
for mu_val in unique_mus:
    ##find which examples have a particular mu value (mu_val)
    mu_loc = np.where(mus == mu_val)[0]
    ##find the labels and prediction of observables which correspond to these examples
    obs_label_mu_val = labels[mu_loc]
    obs_pred_mu_val = net_outputs[mu_loc]
    for target_i, target_name in enumerate(target_attributes):
        
        obs_label_mu = np.array(labels[mu_loc,target_i])
        obs_pred_mu = np.array(net_outputs[mu_loc,target_i])
        
        obs_mu_dict[target_name]["label"].append(obs_label_mu)
        obs_mu_dict[target_name]["pred"].append(obs_pred_mu)

        
        
        
for target_i, target_name in enumerate(target_attributes):
    obs_mu_dict[target_name]["label"] = np.array(obs_mu_dict[target_name]["label"])
    obs_mu_dict[target_name]["pred"] = np.array(obs_mu_dict[target_name]["pred"])

In [ ]:
plt.rcParams['figure.figsize'] = [10, 10]

plot_target = "phi2"

obs_label = np.array(obs_mu_dict[plot_target]["label"])
obs_pred = np.array(obs_mu_dict[plot_target]["pred"])

mean_obs_label = np.zeros(len(obs_label))
std_obs_label = np.zeros(len(obs_label))

mean_obs_pred = np.zeros(len(obs_pred))
std_obs_pred = np.zeros(len(obs_pred)) 

for i in range(len(obs_label)):
    mean_obs_label[i] = np.mean(obs_label[i])
    std_obs_label[i] = np.std(obs_label[i])
    
    mean_obs_pred[i] = np.mean(obs_pred[i])
    std_obs_pred[i] = np.std(obs_pred[i])
    
#plt.scatter(unique_mus, mean_obs_label, c="g")
#plt.scatter(unique_mus, mean_obs_pred, c="r")
    
plt.errorbar(unique_mus, mean_obs_label, yerr=std_obs_label, color="g", label="ϕ² label")
plt.errorbar(unique_mus, mean_obs_pred, yerr=std_obs_pred, color="r", label="ϕ² prediction")
plt.legend(loc="upper left")
plt.savefig("phi2_mu")

In [ ]:
plt.rcParams['figure.figsize'] = [10, 10]

plot_target = "n"

obs_label = np.array(obs_mu_dict[plot_target]["label"])
obs_pred = np.array(obs_mu_dict[plot_target]["pred"])

mean_obs_label = np.zeros(len(obs_label))
std_obs_label = np.zeros(len(obs_label))

mean_obs_pred = np.zeros(len(obs_pred))
std_obs_pred = np.zeros(len(obs_pred)) 

for i in range(len(obs_label)):
    mean_obs_label[i] = np.mean(obs_label[i])
    std_obs_label[i] = np.std(obs_label[i])
    
    mean_obs_pred[i] = np.mean(obs_pred[i])
    std_obs_pred[i] = np.std(obs_pred[i])
    
#plt.scatter(unique_mus, mean_obs_label, c="g")
#plt.scatter(unique_mus, mean_obs_pred, c="r")
    
plt.errorbar(unique_mus, mean_obs_label, yerr=std_obs_label, color="g", label="n label")
plt.errorbar(unique_mus, mean_obs_pred, yerr=std_obs_pred, color="r", label="n prediction")
plt.legend(loc="upper left")
plt.savefig("n_mu")

In [ ]:
plt.rcParams['figure.figsize'] = [10, 10]

target_i = 0

x_max = np.max(labels[:,target_i])
x_min = np.min(labels[:,target_i])


#plt.scatter(labels[:,target_i], net_outputs[:,target_i], c="r", label="model")
plt.scatter(labels[:,target_i], net_outputs[:,target_i], c=mus, label="model")
plt.plot([x_min, x_max], [x_min, x_max], linestyle="--", color="b", label="(ideal)")
plt.xlabel("ϕ² label")
plt.ylabel("ϕ² predicted")
plt.legend(loc="upper left")
#plt.show()
plt.savefig("phi2_label_pred") 

In [ ]:
plt.rcParams['figure.figsize'] = [20, 20]

target_i = 0

fig, ax = plt.subplots([len(unique_mus)])

target_labels = labels[:,target_i]
target_net_outputs[:,target_i]

for mu_i, mu_val in enumerate(unique_mus):
    
    plot_indices = np.argwhere(unique_mus == mu_val)

    plot_target_labels = target_labels[plot_indices]
    plot_target_net_outputs = target_net_outputs[plot_indices]
    
    ax[mu_i] = plt.scatter(plot_target_labels, plot_target_net_outputs)
    ax[mu_i] = plt.plot([x_min, x_max], [x_min, x_max], linestyle="--", color="b", label="(ideal)")

In [ ]:
plt.rcParams['figure.figsize'] = [10, 10]

target_i = 1

x_max = np.max(labels[:,target_i])
x_min = np.min(labels[:,target_i])


#plt.scatter(labels[:,target_i], net_outputs[:,target_i], c="r", label="model")
plt.scatter(labels[:,target_i], net_outputs[:,target_i], c=mus, label="model")
plt.plot([x_min, x_max], [x_min, x_max], linestyle="--", color="b", label="(ideal)")
plt.xlabel("n label")
plt.ylabel("n predicted")
plt.legend(loc="upper left")
#plt.show()
plt.savefig("n_label_pred") 